# Shuttle through 10 µm QuBus

Now that the expeimental evidance of long-range shuttling approaches our ultimate goal, it is time to polish previous work.

## Task 1: Simplify the piece-weised pulse
> ### Issue:
> the measurement window of each piece-weised level yields large amount of dataset due to the high sample rate of AWG (i.e. 192/12.5 MHz = 15.36 µs). It is overkilled for loading or reading a single electron tunneling as it is rather slow i.e. in ms range, which as a result generates thousands data points for a tunnel event.

> ### Solutions: 
> From my experience, a time resolution of hundreds µs is sufficient to resolve a single blip due to tunneling. 
> 1. Making simple loading/reading pulse by `TablePT` or `ConstantPT` and let alazar card downsample data as intended.
>> #### Pridictable problems:
>> 1. pulse issue: ConstantPT has inconsistent usage of `Set` and `frozenset` --> Issue: 712. -- DONE
>> 2. atsaverage issue: how should I downsample data with given resolution? How to define mask properly?
>> 3. MATLAB issue: the matlab interface will be buggy for sure. Maybe `conf_seq` has already implemented similar functionalities?
> 2. Making more ineligent pulse as a physist? Just wrapping piece-weised pulses by one more `RepetitionPT` where `measurement_window` can be defined by a given parameter?
>> Other than an un-readable pulse that may potentially make analysis difficult, I don't see any problems by now...

> Others?

Anyway, try to avoid using `PointPT` first.


In [2]:
import sympy
from qupulse.pulses import ConstantPT, RepetitionPT

n_segment, sample_rate = sympy.sympify('n_segment, sample_rate')
t_segment = n_segment / sample_rate
# for HDAWG, n_segment = 192... It is a defined constant MIN_WAVEFORM_LEN = 192 in qupulse.hardware.awg.zihdawg.py

segment = ConstantPT(t_segment, {k : 'v_hold_'+ k for k in 'ABCDEFGH'})
body = RepetitionPT(segment, 't_res // t_segment', measurements=[('M', 0, 't_res')]) # defines resolutioin
section = RepetitionPT(body, 't_hold // t_res') # defines hold on time at each voltage level


Now make 3-level square pulse for loading or reading. Maybe extend it to have a user defined section number.

In [3]:
from qupulse.pulses import MappingPT

# but why do I define measurement window by qupulse?
# I know MappingPT can do that, but why necessary?

n_segment/sample_rate
{'D', 'E', 'B', 'A', 'F', 'C', 'G', 'H'}
frozenset({'n_segment', 'sample_rate', 'V_hold_C', 'V_hold_B', 'V_hold_A', 'V_hold_F', 'V_hold_H', 'V_hold_D', 'V_hold_E', 'V_hold_G'})
[]


In [3]:
print(body)
print(body.defined_channels)
print(body.parameter_names)
print(body._measurement_windows)

RepetitionPulseTemplate: <floor(t_res/t_segment)> times <ConstantPulseTemplate()>
{'D', 'E', 'H', 'F', 'G', 'A', 'B', 'C'}
{'sample_rate', 'V_hold_B', 't_segment', 'V_hold_C', 'V_hold_H', 'V_hold_E', 'V_hold_F', 'n_segment', 'V_hold_A', 't_res', 'V_hold_G', 'V_hold_D'}
[('M', ExpressionScalar(0), ExpressionScalar('t_res'))]


### Task 2: Tune both sides:
